# Import all the libraries

In [41]:
import pandas as pd                 # for working with data in tables
import numpy as np                  # for numerical operations
import re                           # for regular expressions
import nltk                         # for natural language processing
from nltk.corpus import stopwords   # for removing stopwords
from nltk.stem import WordNetLemmatizer  # for lemmatization
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  # for creating document-term matrices
from sklearn.model_selection import train_test_split   # for splitting data into training and testing sets
from sklearn.naive_bayes import MultinomialNB           # for training Naive Bayes classifier

# TfidfVectorizer Explanation

TfidfVectorizer is a technique used to transform a group of raw text documents into a matrix of numerical features based on their TF-IDF (Term Frequency-Inverse Document Frequency) values. The resulting matrix can be used for various tasks such as text classification, clustering, and information retrieval.

TF-IDF where TF means term frequency, and IDF means Inverse Document frequency.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
text = ['Hello Ebisa here, I love machine learning','Welcome to the Machine learning' ]

In [6]:
vect = TfidfVectorizer()

In [7]:
vect.fit(text)

TfidfVectorizer()

In [8]:
## TF will count the frequency of word in each document. and IDF 
print(vect.idf_)

[1.40546511 1.40546511 1.40546511 1.         1.40546511 1.
 1.40546511 1.40546511 1.40546511]


In [9]:
print(vect.vocabulary_)

{'hello': 1, 'ebisa': 0, 'here': 2, 'love': 4, 'machine': 5, 'learning': 3, 'welcome': 8, 'to': 7, 'the': 6}


### A words which is present in all the data, it will have low IDF value. With this unique words will be highlighted using the Max IDF values.

In [10]:
example = text[0]
example

'Hello Ebisa here, I love machine learning'

In [11]:
example = vect.transform([example])
print(example.toarray())

[[0.44665616 0.44665616 0.44665616 0.31779954 0.44665616 0.31779954
  0.         0.         0.        ]]


### Here, 0 is present in the which indexed word, which is not available in given sentence.

## PassiveAggressiveClassifier

#### Passive approach involves retaining the model if it correctly classifies the data. On the other hand, an aggressive approach involves updating the model if it incorrectly classifies the data to adjust to this misclassified example.

Passive-Aggressive algorithms are commonly utilized for large-scale learning and fall under the category of 'online-learning algorithms'. In online machine learning, data is received sequentially, and the model is updated incrementally instead of using the whole dataset at once. This is advantageous when handling massive amounts of data that may be too computationally expensive to process all at once. To put it simply, online-learning algorithms receive a training example, adjust the classifier, and then discard the example.

## Starting the work

In [12]:
import os
os.chdir("C:/Users/Ebi/Desktop/Fake_News_Detection-Proj")

In [13]:
import pandas as pd

In [14]:
dataframe = pd.read_csv('news.csv')
dataframe.head()

Unnamed: 0                                              title  \
0        8476                       You Can Smell Hillary’s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL

In [15]:
x = dataframe['text']
y = dataframe['label']

In [16]:
x

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object

In [17]:
y

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [19]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [20]:
y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [21]:
tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

* max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
* max_df = 25 means "ignore terms that appear in more than 25 documents".

In [22]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [23]:
y_pred = classifier.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.53%


In [24]:
cf = confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])
print(cf)

[[573  42]
 [ 40 612]]


In [25]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

In [26]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

['REAL']


In [27]:
fake_news_det("""Go to Article 
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.  """)

['FAKE']


In [28]:
import pickle
pickle.dump(classifier,open('model.pkl', 'wb'))

In [29]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [30]:
def fake_news_det1(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    print(prediction)

In [31]:
fake_news_det1("""Go to Article 
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.  """)

['FAKE']


In [32]:
fake_news_det1("""U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.""")

['REAL']


In [33]:
fake_news_det('''U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.''')

['REAL']
